In [110]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from sklearn.metrics import r2_score

from mlscorecheck.auc import exponential_fitting, exponential_fitting2

In [111]:
input_label = 'aggregated'

input_file = f'raw-{input_label}-20k.csv'
input_file2 = f'raw-{input_label}-20k-b.csv'
output_file = f'fit-{input_label}-20k.csv'

In [112]:
def fit_curve(row, values, fracs):
    if values == 'fprs':
        values = row[values]
        fracs = row[fracs]
    else:
        values = row[values]
        fracs = row[fracs]
    p = np.logspace(-3, 3, 3000)
    err = np.abs(values[:, None]**p - 1 + (1 - fracs)[:, None]**p)
    err = np.mean(err, axis=0)
    exp = p[np.argmin(err)]

    pred = (1 - (1 - fracs)**exp)**(1/exp)

    r2 = r2_score(values, pred)

    return (r2, exp, 0)

In [113]:
#data = pd.concat([pd.read_csv(input_file), pd.read_csv(input_file2)])
data = pd.read_csv(input_file)

In [114]:
data['fprs'] = data['fprs'].apply(lambda x: np.array(eval(x)))
data['tprs'] = data['tprs'].apply(lambda x: np.array(eval(x)))

data['fracs'] = (data['fprs'] * data['n'] + data['tprs'] * data['p']) / (data['n'] + data['p'])

In [115]:
data['r2_fpr'] = data.apply(lambda row: fit_curve(row, 'fprs', 'fracs'), axis=1)
data['exp_fpr'] = data['r2_fpr'].apply(lambda x: x[1])
data['fit_mode_fpr'] = data['r2_fpr'].apply(lambda x: x[2])
data['r2_fpr'] = data['r2_fpr'].apply(lambda x: x[0])

data['r2_tpr'] = data.apply(lambda row: fit_curve(row, 'tprs', 'fracs'), axis=1)
data['exp_tpr'] = data['r2_tpr'].apply(lambda x: x[1])
data['fit_mode_tpr'] = data['r2_tpr'].apply(lambda x: x[2])
data['r2_tpr'] = data['r2_tpr'].apply(lambda x: x[0])

In [116]:
data.columns

Index(['p', 'n', 'k', 'dataset', 'classifier', 'classifier_params', 'best_acc',
       'best_sens', 'best_spec', 'best_acc_train', 'best_sens_train',
       'best_spec_train', 'acc', 'sens', 'spec', 'auc', 'acc_train',
       'sens_train', 'spec_train', 'auc_train', 'n_nodes', 'n_nodes_train',
       'avg_n_nodes', 'avg_n_nodes_train', 'fprs', 'tprs', 'fprs_train',
       'tprs_train', 'fracs', 'r2_fpr', 'exp_fpr', 'fit_mode_fpr', 'r2_tpr',
       'exp_tpr', 'fit_mode_tpr'],
      dtype='object')

In [117]:
data['fprs'] = data['fprs'].apply(lambda x: str(x.tolist()))
data['tprs'] = data['tprs'].apply(lambda x: str(x.tolist()))
data['fracs'] = data['fracs'].apply(lambda x: str(x.tolist()))

In [118]:
data.to_csv(output_file, index=False)